In [3]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [4]:
data= pd.read_csv("data_test.csv",encoding='utf-8')

In [5]:
data['talk_id'] = data.index +1

In [6]:
# checking number of null cells
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isnull.html
pd.isnull(data).sum()

# find that there are talks with no speaker_occupation recorded

comments              0
description           0
duration              0
event                 0
film_date             0
languages             0
main_speaker          0
name                  0
num_speaker           0
published_date        0
ratings               0
related_talks         0
speaker_occupation    6
tags                  0
title                 0
url                   0
views                 0
talk_id               0
dtype: int64

In [7]:
# checking number of zero cells
data.astype(bool).sum(axis=0)

# find that there are talks with no language

comments              2550
description           2550
duration              2550
event                 2550
film_date             2550
languages             2464
main_speaker          2550
name                  2550
num_speaker           2550
published_date        2550
ratings               2550
related_talks         2550
speaker_occupation    2550
tags                  2550
title                 2550
url                   2550
views                 2550
talk_id               2550
dtype: int64

In [8]:
# check what's the value & change Null to "unknown"
# use iterrows to loop through dataframe: https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas
print("set speaker occupation to Unknown for talks without the data")

# replace row values with conditions met 
# method1: https://stackoverflow.com/questions/36909977/update-row-values-where-certain-condition-is-met-in-pandas
data.loc[pd.isnull(data['speaker_occupation']),['speaker_occupation']] = "Unknown"

# for index,row in data.iterrows():
#     if pd.isnull(row['speaker_occupation']):
# #         print(row['title'])

# # method 2: https://stackoverflow.com/questions/13842088/set-value-for-particular-cell-in-pandas-dataframe-using-index/38467449
#         data.at[index,'speaker_occupation'] = 'Unknown'

# question: why is the following not working
# answer: the rows returned by iterrows() are no longer connected with the original rows(/deepcopy)：https://stackoverflow.com/questions/25478528/updating-value-in-iterrow-for-pandas
#         thus, must use index     
#         row['speaker_occupation']="Unknown"   
    
# check null values again
pd.isnull(data).sum()
# 

set speaker occupation to Unknown for talks without the data


comments              0
description           0
duration              0
event                 0
film_date             0
languages             0
main_speaker          0
name                  0
num_speaker           0
published_date        0
ratings               0
related_talks         0
speaker_occupation    0
tags                  0
title                 0
url                   0
views                 0
talk_id               0
dtype: int64

In [9]:
# check some of the talks here and find that either they are just performances or the data is just missing
# for index,row in data.iterrows():
#     if row['languages'] == 0:
#         print((row['title'],row['speaker_occupation']))

#https://stackoverflow.com/questions/15360925/how-to-get-the-first-column-of-a-pandas-dataframe-as-a-series
no_language=data.loc[data['languages']==0,['talk_id']].iloc[:,0]
# set them to 1 as body language is also a language
data.loc[data['languages']==0,['languages']] = 1

# double check
data.astype(bool).sum(axis=0)

comments              2550
description           2550
duration              2550
event                 2550
film_date             2550
languages             2550
main_speaker          2550
name                  2550
num_speaker           2550
published_date        2550
ratings               2550
related_talks         2550
speaker_occupation    2550
tags                  2550
title                 2550
url                   2550
views                 2550
talk_id               2550
dtype: int64

In [10]:
for index,row in data.iterrows():
    tag_num = len(ast.literal_eval(row['tags']))
    data.at[index,'tag_num'] = tag_num
    
data.head()
# find that the values in tag_num column are float

# change the value type to int:https://stackoverflow.com/questions/21291259/convert-floats-to-ints-in-pandas
data['tag_num']=data['tag_num'].astype(int)


In [11]:
def categorizeTitle(title):
    '''
    this function categorizes titles based on whether it is in the form of a question or statement
titles in the form of questions start with 5W1H or Do/Does/Did/Have
the function returns "question" or "statement" and the title in the standard format 
(Captialize the first word and add ? for questions and . for statements)
    '''
    category = ""
    determinants = ['Who', 'What', 'When', 'Where',
                'Why', 'How', 'Does', 'Do', 'Did', 'Have']
    title = title.capitalize()

    if title.split()[0] in determinants:
        category = "question"
        if title[-1] != "?":
            title += "?"
    else:
        category = "statement"
        if title[-1] != ".":
            title += "."

    return category, title

In [12]:
# standardize Title format and add a column of title_type
for index, row in data.iterrows():
    category,title = categorizeTitle(row['title'])
    data.at[index,'title']=title
    data.at[index,'title_type']=category

In [13]:
# convert unix timestamp to readable datatime: https://stackoverflow.com/questions/34883101/pandas-converting-row-with-unix-timestamp-in-milliseconds-to-datetime
data['film_date'] = pd.to_datetime(data['film_date'],unit='s')
data['published_date'] = pd.to_datetime(data['published_date'],unit='s')

# convert duration to minutes
data['duration'] = round(data['duration']/60,2)


In [14]:
# first find all the rating attributes
all_attr = set()
for index,row in data.iterrows():
    talk_ratings = ast.literal_eval(row['ratings'])
    for indiv_rating in talk_ratings:
        all_attr.add(indiv_rating['name'])

# create a separate dataframe for all ratings for all talks for analysi
rating_count = {'talk_id':[]}
for index,row in data.iterrows():
    talk_ratings = ast.literal_eval(row['ratings'])
    rating_count['talk_id'].append(row['talk_id'])
    for rating in talk_ratings:
        if rating['name'] not in rating_count:
            rating_count[rating['name']]=[rating['count']]
        else:
            rating_count[rating['name']].append(rating['count'])
rating_df = pd.DataFrame(rating_count)
# rating_df.to_csv('rating_df.csv')
rating_df

,talk_id,Funny,Beautiful,Ingenious,Courageous,Longwinded,Confusing,Informative,Fascinating,Unconvincing,Persuasive,Jaw-dropping,OK,Obnoxious,Inspiring
0,1,19645,4573,6073,3253,387,242,7346,10581,300,10704,4439,1174,209,24924
1,2,544,58,56,139,113,62,443,132,258,268,116,203,131,413
2,3,964,60,183,45,78,27,395,166,104,230,54,146,142,230
3,4,59,291,105,760,53,32,380,132,36,460,230,85,35,1070
4,5,1390,942,3202,318,110,72,5433,4606,67,2542,3736,248,61,2893
5,6,1102,706,397,721,285,301,1038,1350,377,2423,669,441,335,5211
6,7,2484,277,320,477,193,72,226,316,225,311,61,308,290,438
7,8,9,98,195,34,26,25,121,153,26,37,57,63,5,175
8,9,60,145,211,562,177,70,623,313,283,764,60,154,83,441
9,10,26,767,116,476,306,111,304,459,838,1543,209,245,371,2556


In [15]:
# check number of talks with each rating
rating_df.astype(bool).sum(axis=0)

talk_id         2550
Funny           2477
Beautiful       2543
Ingenious       2546
Courageous      2541
Longwinded      2485
Confusing       2441
Informative     2537
Fascinating     2549
Unconvincing    2507
Persuasive      2543
Jaw-dropping    2544
OK              2541
Obnoxious       2441
Inspiring       2550
dtype: int64

In [16]:
# find the category of top rating for each talk
# find the highest rating for each talk: https://stackoverflow.com/questions/29919306/find-the-column-name-which-has-the-maximum-value-for-each-row
# convert a series obj to a data frame: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.to_frame.html
# count number of occurences of each value: https://stackoverflow.com/questions/22391433/count-the-frequency-that-a-value-occurs-in-a-dataframe-column
highest_rating= rating_df.drop(['talk_id'],axis=1).idxmax(axis=1).to_frame()
highest_rating.columns=['rating']
highest_rating['rating'].value_counts()

Inspiring       877
Informative     726
Fascinating     263
Funny           163
Beautiful       153
Ingenious       103
Courageous       85
Persuasive       83
Jaw-dropping     53
Unconvincing     23
Longwinded        8
OK                7
Obnoxious         4
Confusing         2
Name: rating, dtype: int64

In [17]:
# standardization of rating values to the % of total rating received, so that we can compare ratings between talks from an equal footing
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sum.html
# substract cols: https://stackoverflow.com/questions/48350850/subtract-two-columns-in-dataframe
rating_df['total'] = rating_df.sum(axis=1)
rating_df['total'] -= rating_df['talk_id'] 
rating_ratio_df = rating_df.copy(deep=True)

for attr in all_attr:
    rating_ratio_df[attr] = rating_ratio_df[attr] / rating_ratio_df['total']*100
rating_ratio_df.to_csv('rating_ratio_df.csv')

In [18]:
# https://stackoverflow.com/questions/22391433/count-the-frequency-that-a-value-occurs-in-a-dataframe-column

highest_rating= rating_ratio_df.drop(['talk_id','total'],axis=1).idxmax(axis=1).to_frame()
highest_rating.columns=['rating']
highest_rating['rating'].value_counts()

Inspiring       877
Informative     726
Fascinating     263
Funny           163
Beautiful       153
Ingenious       103
Courageous       85
Persuasive       83
Jaw-dropping     53
Unconvincing     23
Longwinded        8
OK                7
Obnoxious         4
Confusing         2
Name: rating, dtype: int64

In [19]:
rating_corr = rating_ratio_df.drop(['talk_id','total'],axis=1).copy(deep=True)
rating_corr['duration'] = data['duration']
rating_corr['languages'] = data['languages']
rating_corr['views'] = data['views']
rating_corr['comments'] = data['comments']
rating_corr['tag_num']=data['tag_num']

rating_corr = rating_corr.corr()
rating_corr.to_csv('rating_corr.csv')

In [20]:
# check if talks high on beautiful are performances?

beautiful_talk_id = rating_ratio_df.sort_values(by=['Beautiful'],axis=0,ascending=0).head(20)["talk_id"]

pd.Series(list(set(beautiful_talk_id).intersection(set(no_language))))

# => Out of 20 top beatiful talks, 5 are performances

# https://stackoverflow.com/questions/43979514/pandas-columns-from-another-dataframe-if-value-is-in-another-column
# data.loc[data['talk_id'].isin(beautiful_talk_id),['talk_id','title']]

0     136
1    2408
2     238
3     210
4     116
dtype: int64

In [21]:
# regression data
regr_data = rating_ratio_df.copy(deep=True)
regr_data['views']=data['views']
regr_data['duration'] = data['duration']
regr_data['languages'] = data['languages']
regr_data['views'] = data['views']
regr_data['comments'] = data['comments']
regr_data['tag_num']=data['tag_num']

In [22]:
# checking if higher duration will lead to higher longwinded score
Y = regr_data['Longwinded']
X = regr_data['duration']
X = sm.add_constant(X)
model = sm.OLS(Y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Longwinded   R-squared:                       0.071
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     195.2
Date:                Sat, 16 Mar 2019   Prob (F-statistic):           8.49e-43
Time:                        19:53:39   Log-Likelihood:                -5813.0
No. Observations:                2550   AIC:                         1.163e+04
Df Residuals:                    2548   BIC:                         1.164e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5337      0.114      4.696      0.000       0.311       0.757
duration       0.1050      0.008     13.971      0.000       0.090       0.120
==============================================================================
Omnibus:                     1791.271   Durbin-Watson:                   1.842
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            31094.670
Skew:                           3.150   Prob(JB):                         0.00
Kurtosis:                      18.905   Cond. No.                         36.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [2]:
# fig, ax = plt.subplots(figsize=(10, 5))
# ax.scatter(X,Y, alpha=0.5, color='orchid')
# fig.tight_layout(pad=2)
# ax.grid(True)

In [24]:
tag_count={}
for index,row in data.iterrows():
    tags = ast.literal_eval(row['tags'])
    for tag in tags:
        if tag in tag_count:
            tag_count[tag] +=1
        else:
            tag_count[tag] = 1

tag_count_df = pd.Series(tag_count).to_frame('talk_count')
tag_count_df.to_csv('tag_count_df.csv')
# sort dataframe: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html
tag_count_df.sort_values(by=['talk_count'],ascending=False)

,talk_count
technology,727
science,567
global issues,501
culture,486
TEDx,450
design,418
business,348
entertainment,299
health,236
innovation,229


In [26]:
tag_main_rating_df = tag_count_df.copy(deep=True)
# add multiple empty cols: https://stackoverflow.com/questions/30926670/pandas-add-multiple-empty-columns-to-dataframe
tag_main_rating_df=pd.concat([tag_main_rating_df,pd.DataFrame(columns=list(all_attr))],sort=False)

# find the main rating/characteristics of each tag

for index,row in data.iterrows():
    talk_tags = ast.literal_eval(row['tags'])
    talk_highest_rating = highest_rating.loc[highest_rating['talk_id']==row['talk_id'],'rating'].iloc[0]
    for tag in talk_tags:
        if pd.isnull(tag_main_rating_df.loc[tag][talk_highest_rating]):
            value = 1
        else:
            value = tag_main_rating_df.loc[tag][talk_highest_rating] +1 
        tag_main_rating_df.loc[tag, talk_highest_rating]=value

tag_main_rating_df['talk_count'] = tag_main_rating_df['talk_count'].astype('int64')
tag_main_rating_df = tag_main_rating_df.fillna(value=0)
tag_main_rating_df.to_csv('tag_main_rating_df.csv')
# tag_main_rating_df

KeyError: 'talk_id'